Bibliotecas

In [1]:
import requests
import pandas as pd
import sqlite3

URL

In [2]:
url = 'https://pokeapi.co/api/v2/pokemon/ditto'

Funções

In [ ]:
resposta = requests.get(url)

resposta.status_code

if resposta.status_code == 200:
    data_json = resposta.json()
else:
    printf('Nao foi possivel acessar sua API {url}')

def extrair_tabelas_pokemon(url):
    resposta = requests.get(url)
    dados = resposta.json()
    return dados

dados_pokemon = extrair_tabelas_pokemon(url)

def criar_dataframe(dados_pokemon):
    df_habilidades = pd.DataFrame(dados_pokemon['abilities'])
    df_status = pd.DataFrame(dados_pokemon['stats'])
    df_tipos = pd.DataFrame(dados_pokemon['types'])
    return df_habilidades, df_status, df_tipos

def validar_dados(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT ability, COUNT(*) FROM habilidades GROUP BY ability HAVING COUNT(*) > 1")
    duplicatas = cursor.fetchall()
    if duplicatas:
        print("tem habilidades duplicadas:", duplicatas)

    cursor.execute ("SELECT * FROM stats WHERE NOT base_stat IS INTEGER")
    tipos_invalidos = cursor.fetchall()
    if tipos_invalidos:
        print("Possui tipos de dados invalidos na coluna 'base_stat':", tipos_invalidos)


Criando Tabela

In [43]:
def criar_tabelas(conn):
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS tipos (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    tipo1 TEXT,
    tipo2 TEXT
    )
""")
    
    cursor.execute("""

     CREATE TABLE IF NOT EXISTS habilidades (
     id INTEGER PRIMARY KEY,
     nome_habilidade TEXT,
     pokemon_id INTEGER,
     FOREIGN KEY(pokemon_id) REFERENCES pokemon(id)
     )
""")
    
    cursor.execute("""

    CREATE TABLE IF NOT EXISTS stats (
    id INTEGER PRIMARY KEY,
    pokemon_id INTEGER,
    hp INTEGER,
    ataque INTEGER,
    defesa INTEGER,
    ataque_especial INTEGER,
    defesa_especial INTEGER,
    velocidade INTEGER,
    FOREIGN KEY(pokemon_id) REFERENCES pokemon(id)
)
 """)



conn.commit()


Tratamento dos dados

In [47]:
def inserir_dados(conn, df_tipos, df_status, df_habilidades):
    df_tipos.to_sql('tipos', conn, if_exists='append', index=False)
    df_status.to_sql('status', conn, if_exists='append', index=False)
    df_habilidades.to_sql('habilidades', conn, if_exists='append', index=False)

In [ ]:
df_habilidades.dropna(inplace=True)


Conectar ao banco

In [49]:
def salvar_dados():
    conn = sqlite3.connect('pokedex.db')
    criar_tabelas(conn)

    df_habilidades, df_status, df_tipos = criar_dataframe(dados_pokemon)
    inserir_dados(conn, df_tipos, df_status, df_habilidades)

    validar_dados(conn)

    conn.close()